<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/Database_06_WritingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Writing Data With Rollingstone's Greatest Albums
### Databases Through Pop Culture | Brendan Shea, PhD

In this chapter, we dive into the task of writing data to SQL tables, a fundamental operation in database management. We explore the use of the INSERT INTO statement to add new rows of data to tables, both for single-row and multiple-row insertions. By working with a sample database based on Rolling Stone's Greatest Albums of All Time, we illustrate the process of creating tables, defining constraints, and populating them with data.

The chapter also covers important topics related to data manipulation, such as handling constraint violations, auto-incrementing primary keys, and using subqueries in INSERT statements. We discuss the differences between deleting data using the DELETE statement and dropping entire tables with DROP TABLE.

Moreover, we introduce the concepts of "soft delete" and the UPDATE statement for modifying existing data. We explore the use of triggers to automatically execute specific actions in response to database events and demonstrate how to implement logging mechanisms using triggers.

Finally, we go beyond SQL and explore database scripting using Linux scripts and various programming languages. We discuss the benefits of automating database tasks and leveraging the power of scripting for data insertion, manipulation, and maintenance.

By the end of this chapter, readers will have a solid understanding of writing data to SQL tables, handling constraints, and utilizing advanced techniques for data manipulation and automation.

Learning Outcomes:

1.  Understand the process of creating tables and defining constraints in SQL.
2.  Learn how to insert single and multiple rows of data into tables using the INSERT INTO statement.
3.  Recognize and handle constraint violations during data insertion.
4.  Utilize auto-incrementing primary keys for efficient data insertion.
5.  Employ subqueries in INSERT statements for dynamic data insertion.
6.  Differentiate between deleting data with DELETE and dropping tables with DROP TABLE.
7.  Implement "soft delete" functionality to retain historical data.
8.  Modify existing data using the UPDATE statement.
9.  Create triggers to automate actions based on database events.
10. Implement logging mechanisms using triggers for auditing and monitoring purposes.
11. Explore database scripting using Linux scripts and various programming languages.
12. Understand the use of Object-Relational Modeling (ORM) tools to interact with databases.

Keywords: SQL, INSERT INTO, constraints, DELETE, DROP TABLE, soft delete, UPDATE, triggers, logging, database scripting

## Creating Tables for Rolling Stone's Greatest Albums of All Time
For this chapter, we'll be creating and populating a database based on a small subset of the 2023 version of Rollingstone Magazine's "500 greatest albums of all time".

Before we can start inserting data into our database, we need to create the necessary tables. In this case, we'll create two tables: Artist and Album. Let's define the structure of these tables and include some constraints to ensure data integrity.

In [ ]:
%load_ext sql
%sql sqlite:///greatest.db

In [ ]:
%%sql
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
CREATE TABLE Artist (
    ArtistID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name VARCHAR(100) NOT NULL,
    Country VARCHAR(50),
    Founded INT
    CHECK (Founded >= 1900 AND Founded <= 2023)
);

CREATE TABLE Album (
    AlbumID INTEGER PRIMARY KEY AUTOINCREMENT,
    Title VARCHAR(100) NOT NULL,
    ArtistID INT,
    ReleaseYear INT,
    Genre VARCHAR(50),
    Ranking,
    FOREIGN KEY (ArtistID) REFERENCES Artist(ArtistID),
    CHECK (ReleaseYear >= 1900 AND ReleaseYear <= 2023),
    CHECK (Ranking >= 1 AND Ranking <= 500)
);

--enable foreign key contraints
PRAGMA foreign_keys = ON;

 * sqlite:///greatest.db
Done.
Done.
Done.
Done.
Done.


[]

Let's break down the key aspects of these table definitions:

-   The `Artist` table has an `ArtistID` column as the **primary key**, which uniquely identifies each artist. The `Name` column is marked as `NOT NULL`, ensuring that every artist has a name. We also have `Country` and `Founded` columns to store additional information about the artists.
-   The `Album` table has an `AlbumID` column as the primary key. The `Title` column is marked as `NOT NULL` to ensure that every album has a title. We have a foreign key `ArtistID` that references the `ArtistID` column in the `Artist` table, establishing a relationship between albums and artists. The `ReleaseYear` and `Genre` columns provide additional details about each album.
-  We have specified that the primary keys (`ArtistID` and `AlbumID`) are both `NOT NULL` and `AUTOINCREMENT`. This means they cannot be left blank and that, if they are left blank, SQL will assign the next available integer to them.
-   We include **check constraints** in both tables to enforce certain conditions. In the `Artist` table, we ensure that the `Founded` year is between 1900 and 2023. Similarly, in the `Album` table, we check that the `ReleaseYear` is within the same range.
- Finally, we enable **foreign key constraints**, which means that SQLite will ensure that attempts to update, delete, or insert data that causes problems for this will fail.

By creating these tables with appropriate constraints, we set up a solid foundation for our database. The constraints help maintain data integrity by preventing invalid or inconsistent data from being inserted into the tables.

Now that we have our tables ready, we can start inserting data into them using the `INSERT INTO` statement, which we'll explore in the next section.

## Using INSERT INTO (Single Row)


Now that we have our `Artist` and `Album` tables created, let's explore how to insert a single row of data into each table using the `INSERT INTO` statement.

The basic syntax for inserting a single row is as follows:

```sql
INSERT INTO table_name (column1, column2, ...)
VALUES (value1, value2, ...);
```

Let's insert a single artist into the `Artist` table:

In [ ]:
%%sql
--Insert Marvin Gaye
INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES (1, 'Marvin Gaye', 'United States', 1939);

 * sqlite:///greatest.db
1 rows affected.


[]

In this example, we specify the table name `Artist` and list the columns we want to insert data into (`ArtistID`, `Name`, `Country`, `Founded`). We then provide the corresponding values for each column using the `VALUES` clause. The values are listed in the same order as the columns specified.

Now, let's insert a single album into the `Album` table:

In [ ]:
%%sql
-- Insert What's Going On
INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear,Genre, Ranking)
VALUES (1, 'What''s Going On', 1, 1971, 'Soul', 1);

 * sqlite:///greatest.db
1 rows affected.


[]

In [ ]:
%sql SELECT * FROM Album;

 * sqlite:///greatest.db
Done.


AlbumID,Title,ArtistID,ReleaseYear,Genre,Ranking
1,What's Going On,1,1971,Soul,1


Similarly, we specify the `Album` table and list the columns we want to insert data into. We provide the corresponding values in the `VALUES` clause, ensuring that the `ArtistID` matches the `ArtistID` of the artist we inserted earlier.

It's important to note that when inserting data, we need to provide values for all columns that are marked as `NOT NULL` or do not have a default value defined. If we omit a column that allows `NULL` values or has a default value, the database will automatically assign the appropriate value.

By using the `INSERT INTO` statement, we can easily add single rows of data to our tables. This is particularly useful when we have specific values for each column and want to insert them one at a time.

In the next section, we'll explore how to insert multiple rows of data in a single statement, which is more efficient when dealing with larger datasets.

## Using INSERT INTO (Multiple Rows)

Inserting data one row at a time can be inefficient when you have a large number of records to insert. Fortunately, SQL allows you to insert multiple rows of data in a single `INSERT INTO` statement. This is achieved by specifying multiple sets of values in the `VALUES` clause.

Let's insert the top 10 albums from Rolling Stone's Greatest Albums of All Time list into our database. First, we'll insert the artists:

In [ ]:
%%sql
INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES
(2, 'The Beach Boys', 'United States', 1961),
(3, 'Joni Mitchell', 'Canada', 1964),
(4, 'Stevie Wonder', 'United States', 1961),
(5, 'Nirvana', 'United States', 1987),
(6, 'Fleetwood Mac', 'United Kingdom', 1967),
(7, 'Prince', 'United States', 1975),
(8, 'Bob Dylan', 'United States', 1961),
(9, 'Lauryn Hill', 'United States', 1988),
(10, 'The Beatles', 'United Kingdom', 1960),
(11, 'Radiohead', 'United Kingdom', 1985),
(12, 'Kendrick Lamar', 'United States', 2003),
(13, 'Public Enemy', 'United States', 1985),
(14, 'The Rolling Stones', 'United Kingdom', 1962),
(15, 'Aretha Franklin', 'United States', 1956),
(16, 'Michael Jackson', 'United States', 1964),
(17, 'Kanye West', 'United States', 1996),
(19, 'The Clash', 'United Kingdom', 1976);

 * sqlite:///greatest.db
17 rows affected.


[]

Now, let's insert the corresponding albums:

In [ ]:
%%sql
INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear, Genre, Ranking)
VALUES
(2, 'Pet Sounds', 2, 1966, 'Rock',2),
(3, 'Blue', 3, 1971, 'Folk',3),
(4, 'Songs in the Key of Life', 4, 1976, 'Soul',4),
(5, 'Nevermind', 5, 1991, 'Grunge',5),
(6, 'Rumours', 6, 1977, 'Soft Rock',6),
(7, 'Purple Rain', 7, 1984, 'Pop',7),
(8, 'Blood on the Tracks', 8, 1975, 'Folk Rock',8),
(9, 'The Miseducation of Lauryn Hill', 9, 1998, 'Hip Hop',9),
(10, 'Abbey Road', 10, 1969, 'Rock',10),
(11, 'Revolver', 10, 1966, 'Rock',11),
(12, 'Thriller', 16, 1982, 'Pop',12),
(13, 'I Never Loved a Man the Way I Love You', 15, 1967, 'Soul',13),
(14, 'Exile on Main Street', 14, 1972, 'Rock',14),
(15, 'It Takes a Nation of Millions to Hold Us Back', 13, 1988, 'Hip Hop',15),
(16, 'London Calling', 19, 1979, 'Punk',16),
(17, 'My Beautiful Dark Twisted Fantasy', 17, 2010, 'Hip Hop',17),
(18, 'Highway 61 Revisited', 8, 1965, 'Folk Rock',18),
(19, 'To Pimp a Butterfly', 12, 2015, 'Hip Hop',19),
(20, 'Kid A', 11, 2000, 'Electronic',20);

 * sqlite:///greatest.db
19 rows affected.


[]

In these examples, we use a single `INSERT INTO` statement for each table, but we provide multiple sets of values separated by commas. Each set of values represents a new row to be inserted into the table.

By inserting multiple rows at once, we can significantly reduce the number of statements required and improve the efficiency of our data insertion process.

It's important to ensure that the number of values matches the number of columns specified and that the values adhere to any constraints defined on the table. If there are any violations, such as trying to insert a duplicate primary key value or violating a check constraint, the entire `INSERT INTO` statement will fail, and no rows will be inserted.

In the next section, we'll explore what happens when constraints are violated and how to handle such situations.

## What Happens When Constraints Are Violated?

Constraints are essential for maintaining data integrity in a database. They enforce rules and restrictions on the data that can be inserted into tables. When attempting to insert data that violates these constraints, SQL will raise an error and prevent the insertion from occurring. Let's explore some common constraint violations and their consequences.

### Problems with Primary Keys

A primary key uniquely identifies each row in a table. It ensures that no two rows have the same primary key value. If we attempt to insert a row with a primary key value that already exists in the table, SQL will raise a primary key constraint violation error.

For example, let's try to insert an artist with an existing `ArtistID`:

In [ ]:
%%sql
INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES (1, 'Pink Floyd', 'United Kingdom', 1962);

 * sqlite:///greatest.db
(sqlite3.IntegrityError) UNIQUE constraint failed: Artist.ArtistID
[SQL: INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES (1, 'Pink Floyd', 'United Kingdom', 1962);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Since we already have an artist with an `artist_id` of 1 this fails with an error message. This is because primary keys must be UNIQUE.

### Violating CHECK Constraints

CHECK constraints allow us to specify conditions that the data must satisfy before it can be inserted into a table. If we attempt to insert data that violates a CHECK constraint, SQL will raise a constraint violation error.

For example, let's try to insert an album with an invalid `ReleaseYear`:

In [ ]:
%%sql
INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear, Genre)
VALUES (11, 'Future Album', 1, 2028, 'Rock');

 * sqlite:///greatest.db
(sqlite3.IntegrityError) CHECK constraint failed: ReleaseYear >= 1900 AND ReleaseYear <= 2023
[SQL: INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear, Genre)
VALUES (11, 'Future Album', 1, 2028, 'Rock');]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In our Album table, we have a CHECK constraint that ensures the ReleaseYear is between 1900 and 2023. Since 2028 violates this constraint, the insertion will fail with an error message indicating a CHECK constraint violation.

### Violating NOT NULL Constraints

NOT NULL constraints ensure that a column cannot contain a NULL value. If we attempt to insert a row with a NULL value for a column that has a NOT NULL constraint, SQL will raise a constraint violation error.

For example, let's try to insert an artist without specifying a `Name`:

In [ ]:
%%sql
INSERT INTO Artist (ArtistID, Country, Founded)
VALUES (21, 'United States', 1980);

 * sqlite:///greatest.db
(sqlite3.IntegrityError) NOT NULL constraint failed: Artist.Name
[SQL: INSERT INTO Artist (ArtistID, Country, Founded)
VALUES (21, 'United States', 1980);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Since the `Name` column in the `Artist` table has a NOT NULL constraint, this insertion will fail with an error message indicating a NOT NULL constraint violation.

When constraint violations occur, the entire `INSERT INTO` statement is rolled back, and no data is inserted into the table. This ensures that the database remains in a consistent state and maintains data integrity.

To handle constraint violations, you can:

-   Modify the data being inserted to satisfy the constraints.
-   Update the table structure or constraints to accommodate the data.
-   Catch and handle the specific error messages in your application code.

By understanding and properly handling constraint violations, you can ensure that only valid and consistent data is inserted into your database tables.

### Auto-incrementing Primary Keys
As we just saw, SQL will generally produce an error if you fail to provide values for a `NOT NULL` column. One exception to this is for integer primary keys with an `AUTOINCREMENT` option (such as `ArtistId` and `AlbumId`). If you leave this out of an insert statement, SQL will assign the *next available integer as a primary key.

Here's an example:


In [ ]:
%%sql
--Inserting without specifying primary key
INSERT INTO Artist (Name, Country, Founded)
VALUES ('Pink Floyd', 'United Kingdom', 1962);

 * sqlite:///greatest.db
1 rows affected.


[]

In [ ]:
%%sql
SELECT *
FROM Artist
WHERE Name = "Pink Floyd";

 * sqlite:///greatest.db
Done.


ArtistID,Name,Country,Founded
20,Pink Floyd,United Kingdom,1962


## Deleting Data

When working with databases, there may be situations where you need to remove data from tables. In SQL, the `DELETE` statement is used to delete rows from a table based on specified conditions. It's important to understand how to use the `DELETE` statement effectively and handle the deletion of related data to maintain data integrity.

Let's start by inserting some sample data into our `Artist` and `Album` tables. We'll insert a fake artist named "The Terrible Trio" and a few terrible albums associated with this artist.

First, let's insert the fake artist into the `Artist` table:

In [ ]:
%%sql
INSERT INTO Artist (Name, Country, Founded)
VALUES ('The Terrible Trio', 'Nowhere', 2020);

 * sqlite:///greatest.db
1 rows affected.


[]

Now, let's insert some terrible albums. We'll use a subquery to find the ArtistID for the "Terrible Trio."

In [ ]:
%%sql
-- Inserting albums
INSERT INTO Album (Title, ArtistID, ReleaseYear, Genre)
VALUES
    ('Awful Anthems', (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio'), 2005, 'Noise'),
    ('Cringeworthy Chronicles', (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio'), 2010, 'Cacophony'),
    ('Disastrous Ditties', (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio'), 2015, 'Racket');

 * sqlite:///greatest.db
3 rows affected.


[]

Let's assure ourselves that these albums have been added.

In [ ]:
%%sql
SELECT
  al.Title,
  al.AlbumID,
  ar.Name AS Artist,
  ar.ArtistID
FROM
  Album al
  JOIN Artist ar ON al.ArtistID = ar.ArtistID
WHERE ar.Name = 'The Terrible Trio';

 * sqlite:///greatest.db
Done.


Title,AlbumID,Artist,ArtistID
Awful Anthems,21,The Terrible Trio,21
Cringeworthy Chronicles,22,The Terrible Trio,21
Disastrous Ditties,23,The Terrible Trio,21


### Basic DELETE Statement

The basic syntax of the `DELETE` statement is as follows:

```sql
DELETE FROM table_name
WHERE condition;
```

-   `table_name`: The name of the table from which you want to delete rows.
-   `condition`: Specifies the condition that determines which rows will be deleted. If omitted, all rows in the table will be deleted.

For example, to delete the album "Awful Anthems" from the `Album` table, you can use the following statement:

In [ ]:
%%sql
DELETE FROM Album
WHERE Title = 'Awful Anthems';

 * sqlite:///greatest.db
1 rows affected.


[]

In [ ]:
%%sql
--Check that album is deleted
SELECT * FROM Album WHERE Title = 'Awful Anthems';

 * sqlite:///greatest.db
Done.


AlbumID,Title,ArtistID,ReleaseYear,Genre,Ranking


### Deleting Related Data

When deleting data from a table that has related data in other tables, you need to consider the foreign key constraints and how to handle the deletion of related records.

In our example, the `Album` table has a foreign key constraint on the `ArtistID` column that references the `ArtistID` column in the `Artist` table. This means that each album is associated with an artist.

In [ ]:
%%sql
-- This will fail
DELETE FROM Artist
WHERE Name = "The Terrible Trio";

 * sqlite:///greatest.db
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: -- This will fail
DELETE FROM Artist
WHERE Name = "The Terrible Trio";]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In order to deal with with this problem, you have a few different options.



### Manual Deletion
The most straightforward to way to do this deletion is to:
1. Delete the related albums from the Album table first.
2. Second, delete the artist from the Artist table.

```sql
--Delete albums
DELETE FROM Album WHERE ArtistID = (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio');

--Delete artist
DELETE FROM Artist WHERE Name = 'The Terrible Trio';
```

### ON DELETE CASCADE

The `ON DELETE CASCADE` option is used to automatically delete related records from a child table when a record in the parent table is deleted. It ensures data consistency and maintains referential integrity by cascading the delete operation to the associated records.

When you define a foreign key constraint with `ON DELETE CASCADE`, deleting a record from the parent table will automatically delete all the related records in the child table that reference the deleted parent record.

To set up `ON DELETE CASCADE`, you include it in the foreign key constraint definition when creating the child table:

```sql
CREATE TABLE Artist(
  -- define all your columms
  ...
  FOREIGN KEY (ArtistID) REFERENCES Artist(ArtistID) ON DELETE CASCADE
)
```

By adding this line to the foreign key constraint, you instruct the database to cascade the delete operation from the parent table (`Artist`) to the child table (`Album`) when an artist is deleted.

With this option set, you can delete an artist and its related albums using a single DELETE statement:

```sql
DELETE FROM Artist WHERE Name = 'The Terrible Trio';
```

### ON DELETE SET NULL

The `ON DELETE SET NULL` option is used to automatically set the foreign key values in the child table to `NULL` when a record in the parent table is deleted. It allows you to maintain the child records even if the associated parent record is removed.

When you define a foreign key constraint with `ON DELETE SET NULL`, deleting a record from the parent table will set the foreign key values in the child table to `NULL` for the related records.

To set up `ON DELETE SET NULL`, you include it in the foreign key constraint definition when creating the child table:

```sql
CREATE TABLE Artist(
  -- define all your columms
  ...
  FOREIGN KEY (ArtistID) REFERENCES Artist(ArtistID) ON DELETE SET NULL
)

```
By adding this line to the foreign key constraint, you instruct the database to set the `ArtistID` values in the `Album` table to `NULL` when an artist is deleted from the `Artist` table.

### Choosing Between ON DELETE CASCADE, ON DELETE SET NULL, and no action

The choice between `ON DELETE CASCADE` and `ON DELETE SET NULL` depends on your application's requirements and data integrity rules.

-   Use `ON DELETE CASCADE` when you want to ensure that related records in the child table are automatically deleted when a parent record is deleted. This option maintains strict referential integrity and removes all associated data.
-   Use `ON DELETE SET NULL` when you want to keep the related records in the child table even if the parent record is deleted. This option allows you to maintain the child records but sets the foreign key values to `NULL`, indicating that they are no longer associated with a valid parent record.

It's important to consider the implications of each option. `ON DELETE CASCADE` permanently deletes related data, which may not be desirable in all scenarios. `ON DELETE SET NULL` keeps the related records but with a `NULL` foreign key value, which may require additional handling in your application.

If you don't specify either `ON DELETE CASCADE` or `ON DELETE SET NULL`, the default behavior is to restrict the deletion of a parent record if there are related records in the child table. In this case, you would need to manually delete the related records from the child table before deleting the parent record.

### Deleting All Data versus Dropping a Table

As we've seen, the **`DELETE FROM`** command is used to remove rows from a table. This operation can be selective or comprehensive. For example, if you only want to delete rows that meet certain criteria, you use a **`WHERE`** clause with your **`DELETE`** statement. Without a **`WHERE`** clause, **`DELETE`** will remove all rows in the table, but importantly, the table's structure remains untouched.  So, If you decide to remove all albums from the **`albums`** table, you simply omit the **`WHERE`** clause:

```sql
DELETE FROM Albums;
```

This action clears all data from the **`albums`** table but keeps its structure intact for future use. You can still add new albums to it or modify its structure later.

One significant aspect of **`DELETE`** operations is that they are logged row by row in the database's transaction log. This means each row deletion is recorded, allowing for the possibility to undo the deletions if the operation is part of a transaction. However, this logging can make **`DELETE`** operations slower when dealing with a large number of rows.

The **`DROP TABLE`** command, in contrast, is much more drastic. When you execute a **`DROP TABLE`** statement, you remove the entire table from the database. This includes not just the data but the table's structure, its columns, indexes, and any constraints defined on it.

For instance, if you decide that the **`artists`** table is no longer needed and you want to erase it entirely from the database, you would use:

```sql
DROP TABLE Artists;
```

Executing this command means the **`artists`** table is deleted. The table, along with all its data and structure, is permanently removed from the database. Unlike **`DELETE`**, **`DROP TABLE`** does not log individual row deletions because it doesn't process each row; it removes the table as a whole. This makes **`DROP TABLE`** a fast operation but with the significant caveat that it is typically irreversible through standard SQL commands. Once a table is dropped, you cannot simply undo the action unless you have backups or specific database recovery tools in place.

## UPDATE

The `UPDATE` statement in SQL is used to modify existing data in a table. It allows you to change the values of one or more columns in one or more rows based on specified conditions. The basic syntax of the `UPDATE` statement is as follows:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
```

-   `table_name`: The name of the table you want to update.
-   `column1, column2, ...`: The columns you want to modify and their new values.
-   `condition`: Optional. Specifies the condition that determines which rows will be updated. If omitted, all rows in the table will be updated.

Let's look at a concrete example using our `Artist` and `Album` tables.

Suppose we want to update the name of the artist "Bob Dylan" to "Robert Zimmerman".

In [ ]:
%%sql
UPDATE Artist
SET Name = 'Robert Zimmerman'
WHERE Name = 'Bob Dylan';

 * sqlite:///greatest.db
1 rows affected.


[]

In [ ]:
%%sql
-- Confirm our change
SELECT * FROM Artist WHERE Name = 'Robert Zimmerman'

 * sqlite:///greatest.db
Done.


ArtistID,Name,Country,Founded
8,Robert Zimmerman,United States,1961


You can also update multiple columns in a single UPDATE statement by separating the column-value pairs with commas. For example, let's update the Prince's name to "?" and his County to "Minnesota, United States."

In [ ]:
%%sql
UPDATE Artist
SET Name = '?', Country = 'Minnesota, United States'
WHERE Name = 'Prince';

 * sqlite:///greatest.db
1 rows affected.


[]

In [ ]:
%%sql
-- Confirm our change
SELECT * FROM Artist WHERE Name = '?'

 * sqlite:///greatest.db
Done.


ArtistID,Name,Country,Founded
7,?,"Minnesota, United States",1975


It's important to be cautious when using the `UPDATE` statement, especially if you omit the `WHERE` clause. Without a `WHERE` clause, the update will be applied to all rows in the table, which may lead to unintended changes.

To update multiple rows based on a condition, you can use a more complex `WHERE` clause:

In [ ]:
%%sql
UPDATE Album
SET ReleaseYear = ReleaseYear + 1
WHERE ArtistID = 21;

 * sqlite:///greatest.db
2 rows affected.


[]

In [ ]:
%%sql
-- Confirm our change
SELECT * FROM Album WHERE ArtistID = 21;

 * sqlite:///greatest.db
Done.


AlbumID,Title,ArtistID,ReleaseYear,Genre,Ranking
22,Cringeworthy Chronicles,21,2011,Cacophony,None
23,Disastrous Ditties,21,2016,Racket,None


This statement increments the `ReleaseYear` by 1 for all albums associated with the artist having `ArtistID` 21.

Remember to always double-check your `UPDATE` statements and include appropriate `WHERE` conditions to ensure that you are modifying only the intended rows.

The `UPDATE` statement is a powerful tool for modifying existing data in your database tables. It allows you to keep your data up to date and make necessary changes based on specific conditions.

## Introduction to Soft Delete

In database management, there are situations where you may want to keep records even after they are marked as deleted. Instead of permanently removing data from the database, you can implement a "soft delete" approach. Soft delete involves adding a column to the table that indicates whether a record is active or deleted, allowing you to retain historical data while still being able to filter out deleted records when querying the table.

Let's explore the concept of soft delete using our example of the terrible trio artist and their albums.

### Adding a Soft Delete Column

To implement soft delete, we need to add a column to the `Artist` and `Album` tables that represents the deleted status of each record. We'll call this column `IsDeleted` and set its default value to `0` (indicating an active record)

In [ ]:
%%sql
ALTER TABLE Artist ADD COLUMN IsDeleted INTEGER DEFAULT 0;
ALTER TABLE Album ADD COLUMN IsDeleted INTEGER DEFAULT 0;

 * sqlite:///greatest.db
Done.
Done.


[]

These statements add the `IsDeleted` column to the `Artist` and `Album` tables, respectively.

### Soft Deleting Records

Now, let's say we want to soft delete the artist "The Terrible Trio" and their albums. Instead of using the `DELETE` statement, we'll update the `IsDeleted` column to mark the records as deleted.

In [ ]:
%%sql
UPDATE Artist SET IsDeleted = 1 WHERE Name = 'The Terrible Trio';
UPDATE Album SET IsDeleted = 1 WHERE ArtistID = (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio');

 * sqlite:///greatest.db
1 rows affected.
2 rows affected.


[]

The first statement marks the artist "The Terrible Trio" as deleted by setting the `IsDeleted` column to `1`. The second statement marks all the albums associated with "The Terrible Trio" as deleted by setting their `IsDeleted` column to `1`.

### Querying Soft Deleted Records

When querying the `Artist` and `Album` tables, you can filter out the soft deleted records by adding a condition to check the `IsDeleted` column.

In [ ]:
%%sql
SELECT * FROM Artist WHERE IsDeleted = 0;

 * sqlite:///greatest.db
Done.


ArtistID,Name,Country,Founded,IsDeleted
1,Marvin Gaye,United States,1939,0
2,The Beach Boys,United States,1961,0
3,Joni Mitchell,Canada,1964,0
4,Stevie Wonder,United States,1961,0
5,Nirvana,United States,1987,0
6,Fleetwood Mac,United Kingdom,1967,0
7,?,"Minnesota, United States",1975,0
8,Robert Zimmerman,United States,1961,0
9,Lauryn Hill,United States,1988,0
10,The Beatles,United Kingdom,1960,0


### Restoring Soft Deleted Records
One of the advantages of soft delete is the ability to restore deleted records if needed. To restore a soft deleted record, you can update the IsDeleted column back to 0.

In [ ]:
%%sql
UPDATE Artist SET IsDeleted = 0 WHERE Name = 'The Terrible Trio';
UPDATE Album SET IsDeleted = 0 WHERE ArtistID = (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio');

 * sqlite:///greatest.db
1 rows affected.
2 rows affected.


[]

These statements restore the artist "The Terrible Trio" and their associated albums by setting the `IsDeleted` column back to `0`.

### Considerations

When implementing soft delete, keep in mind the following considerations:

-   Soft delete adds an overhead to your queries, as you need to include the `IsDeleted` condition in your `WHERE` clauses to filter out deleted records.
-   Soft deleted records still occupy space in the database, so you need to have a strategy for eventually purging them if necessary.
-   If you have foreign key constraints, you may need to handle soft deletion carefully to maintain data integrity. You can consider using `ON DELETE SET NULL` or `ON UPDATE CASCADE` to manage the relationships between soft deleted records.

Soft delete provides a flexible approach to data deletion, allowing you to retain historical data while still being able to manage deleted records effectivel

In [ ]:
%%sql
-- Let's get rid of our soft delete column
ALTER TABLE Artist DROP COLUMN IsDeleted;
ALTER TABLE Album DROP COLUMN IsDeleted;

 * sqlite:///greatest.db
Done.
Done.


[]

## Triggers

Triggers are special types of stored procedures in SQL that are automatically executed in response to specific events or actions on a table, such as `INSERT`, `UPDATE`, or `DELETE` operations. Triggers allow you to enforce business rules, maintain data integrity, and perform additional actions before or after the triggering event occurs.

In SQLite, you can create triggers using the `CREATE TRIGGER` statement. The basic syntax for creating a trigger is as follows:

```sql
CREATE TRIGGER (IF NOT EXISTS) trigger_name
BEFORE|AFTER INSERT|UPDATE|DELETE ON table_name
BEGIN
  -- Trigger actions
END;
```

-   `trigger_name`: The name you assign to the trigger.
-   `BEFORE|AFTER`: Specifies whether the trigger should be executed before or after the triggering event.
-   `INSERT|UPDATE|DELETE`: Specifies the event that activates the trigger.
-   `table_name`: The name of the table on which the trigger is defined.
-   `BEGIN ... END`: The block of SQL statements that define the trigger actions.

Let's create a concrete example using our `Artist` and `Album` tables.

Suppose, for example, we'd like to keep track of the date each albums was updated to our database.

First, let's add a column to accomodate this:

In [ ]:
%%sql
ALTER TABLE Album ADD COLUMN LastUpdated DATETIME;

 * sqlite:///greatest.db
Done.


[]

Now, let's create trigger.

In [ ]:
%%sql
CREATE TRIGGER IF NOT EXISTS update_album_date
AFTER UPDATE ON Album
BEGIN
  UPDATE Album
  SET LastUpdated = DATETIME('now')
  WHERE AlbumID = NEW.AlbumID;
END;

 * sqlite:///greatest.db
Done.


[]

Finally, let's try it out:

In [ ]:
%%sql
UPDATE Album
SET ReleaseYear = ReleaseYear - 1
WHERE AlbumID > 20;

 * sqlite:///greatest.db
2 rows affected.


[]

And finally, we can see the results.

In [ ]:
%%sql
SELECT * FROM Album WHERE AlbumID > 20;

 * sqlite:///greatest.db
Done.


AlbumID,Title,ArtistID,ReleaseYear,Genre,Ranking,LastUpdated
22,Cringeworthy Chronicles,21,2010,Cacophony,None,2024-06-06 15:30:36
23,Disastrous Ditties,21,2015,Racket,None,2024-06-06 15:30:36


## Logging in Databases

Logging is the process of recording specific events, changes, or actions that occur within a database. Logs serve as an audit trail, allowing database administrators and developers to track and monitor the activities happening in the database. They provide valuable information for debugging, troubleshooting, performance analysis, and security auditing.

Logs can capture various types of information, such as:

-   Data modifications (inserts, updates, deletes)
-   User authentication and access attempts
-   System errors and exceptions
-   Timestamps of when events occurred
-   User or application responsible for the actions

By maintaining logs, you can gain insights into the historical changes in your database, detect suspicious activities, and facilitate data recovery in case of failures or accidental modifications.

Now, let's see how we can use a trigger to implement logging in our database.

### Logging Trigger Example

Suppose we want to create a log table that captures the changes made to the `Artist` table. Whenever an artist's information is updated, we want to record the old and new values, along with the timestamp and the user who made the change.

First, let's create the log table:

In [ ]:
%%sql
DROP TABLE IF EXISTS ArtistLog;
CREATE TABLE ArtistLog (
  LogID INTEGER PRIMARY KEY AUTOINCREMENT,
  ArtistID INTEGER,
  OldName VARCHAR(100),
  NewName VARCHAR(100),
  UpdatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
  UpdatedBy VARCHAR(100)
);

 * sqlite:///greatest.db
Done.
Done.


[]

The `ArtistLog` table has columns to store the `ArtistID`, the old and new values of the `Name` column, the timestamp of the update, and the user who made the change.

Next, let's create the trigger that will populate the log table whenever an update occurs on the `Artist` table:

In [ ]:
%%sql
CREATE TRIGGER IF NOT EXISTS ArtistUpdateTrigger
AFTER UPDATE ON Artist
FOR EACH ROW
BEGIN
  INSERT INTO ArtistLog (ArtistID, OldName, NewName, UpdatedBy)
  VALUES (OLD.ArtistID, OLD.Name, NEW.Name, 'Brendan Shea');
END;

 * sqlite:///greatest.db
Done.


[]

Let's break down the trigger:

-   The trigger is named `ArtistUpdateTrigger`.
-   It is defined as an `AFTER UPDATE` trigger on the `Artist` table, meaning it will be executed after an update operation on the `Artist` table.
-   The `FOR EACH ROW` clause specifies that the trigger will be executed for each row affected by the update.
-   Inside the trigger, we have an `INSERT` statement that inserts a new row into the `ArtistLog` table.
-   The `VALUES` clause captures the `ArtistID`, the old and new values of the `Name` column using the special `OLD` and `NEW` keywords, and the user who made the changes. Here, I've hardcoded this as 'Brendan Shea', but in more complext applications, we could capture the details for each user.

Now, let's test the trigger by updating some artists' names (undoing our previous updates):

In [ ]:
%%sql
UPDATE Artist
SET Name = 'Bob Dylan'
WHERE Name = 'Robert Zimmerman';

UPDATE Artist
SET Name = 'Prince'
WHERE Name = '?';

 * sqlite:///greatest.db
1 rows affected.
1 rows affected.


[]

Now, let's see what our log table looks like:

In [ ]:
%%sql
SELECT * FROM ArtistLog;

 * sqlite:///greatest.db
Done.


LogID,ArtistID,OldName,NewName,UpdatedAt,UpdatedBy
1,8,Robert Zimmerman,Bob Dylan,2024-06-06 15:30:36,Brendan Shea
2,7,?,Prince,2024-06-06 15:30:36,Brendan Shea


The result will show the logged entry with the `ArtistID`, the old and new values of the `Name` column, the timestamp of the update, and the user who made the change.

By using triggers for logging, you can automatically capture and store the changes made to your database tables, providing a valuable audit trail for monitoring, debugging, and historical analysis.

## Inserting Data From CSV Files (Linux/Bash)

In this section, we will explore how to insert data into our database using CSV files and Linux scripts. We will cover the concept of CSV files, introduce Linux scripts, and discuss the benefits of using this approach compared to SQL INSERT statements.

**CSV (Comma-Separated Values)** is a popular file format used for storing and exchanging tabular data. In a CSV file, each line represents a row of data, and the values within each row are separated by commas. CSV files are widely supported by various applications and can be easily created, edited, and processed using spreadsheet software or text editors.

Here's an example of a CSV file named `current_artists.csv` containing information about artists:

In [ ]:
%%writefile current_artists.csv
101,Taylor Swift,United States,2006
102,Ed Sheeran,United Kingdom,2005
103,Billie Eilish,United States,2016
104,BTS,South Korea,2013
105,Ariana Grande,United States,2008
106,Shawn Mendes,Canada,2013
107,Dua Lipa,United Kingdom,2015
108,Post Malone,United States,2013
109,Khalid,United States,2016
110,Camila Cabello,Cuba,2012

Overwriting current_artists.csv


Each line in the CSV file represents an artist, with fields such as the artist ID, name, country, and year they started their career.

### Linux Scripts for Data Insertion

Linux provides a powerful command-line interface and scripting capabilities that allow us to automate tasks and perform operations on files and databases. In this context, we can use Linux scripts to insert data from CSV files into our database.

Here's an example of a Linux script that imports data from a CSV file into an SQLite database:

In [ ]:
%%capture
# first, install sqlite 3 for ubuntu
!sudo apt-get install sqlite3 -q

In [ ]:
# Here is the actual script
!sqlite3 greatest.db ".mode csv" ".import current_artists.csv Artist"

Let's break down the script:

-   `sqlite3` is the command to open the SQLite database.
-   `greatest.db` is the name of the database file.
-   `.mode csv` sets the import mode to CSV.
-   `.import current_artists.csv Artist` imports the data from the `current_artists.csv` file into the `Artist` table of the database.

By executing this script in the Linux terminal, the data from the CSV file will be inserted into the specified table of the database.

Let's now confirm that our data has been inserted sucessfully:

In [ ]:
%%sql
SELECT * FROM Artist WHERE ArtistID > 100;

 * sqlite:///greatest.db
Done.


ArtistID,Name,Country,Founded
101,Taylor Swift,United States,2006
102,Ed Sheeran,United Kingdom,2005
103,Billie Eilish,United States,2016
104,BTS,South Korea,2013
105,Ariana Grande,United States,2008
106,Shawn Mendes,Canada,2013
107,Dua Lipa,United Kingdom,2015
108,Post Malone,United States,2013
109,Khalid,United States,2016
110,Camila Cabello,Cuba,2012


## Database Scripting: Beyond SQL

In the previous section, we explored how to insert data from CSV files into a database using Linux scripts, specifically Bash. This is just one example of database scripting. In this section, we will expand our understanding of database scripting and explore different languages that are commonly used for this purpose.

### Client-Side vs. Server-Side Scripting

When it comes to database scripting, there are two main approaches: client-side scripting and server-side scripting.

1.  **Client-Side Scripting**: Client-side scripting refers to scripts that run on the client machine, such as a user's computer. These scripts interact with the database by sending requests to the database server and processing the responses. Client-side scripting is commonly used for tasks like data analysis, reporting, and automated data entry.
2.  **Server-Side Scripting**: Server-side scripting, on the other hand, involves scripts that run directly on the database server. These scripts have direct access to the database and can perform tasks like data manipulation, database maintenance, and automated backups. Server-side scripting is often used for tasks that require high performance or need to be scheduled and executed on the server.

### Popular Languages for Database Scripting

While SQL is the standard language for interacting with databases, there are several other languages that are commonly used for database scripting. Let's take a look at a few of them:

1.  **Bash**: Bash (Bourne Again Shell) is a Unix shell and command language. It is the default shell on many Linux distributions and is widely used for scripting and automation. Bash scripts can be used to interact with databases, execute SQL commands, and perform file operations, as we saw in the previous section.
2.  **Python**: Python is a versatile and beginner-friendly programming language that has extensive support for database operations. With libraries like SQLite3, MySQL Connector, and psycopg2 (for PostgreSQL), Python scripts can connect to databases, execute SQL queries, and manipulate data. Python's readability and simplicity make it a popular choice for database scripting.
3.  **PowerShell**: PowerShell is a task automation and configuration management framework developed by Microsoft. It is commonly used in Windows environments for scripting and administration tasks. PowerShell provides cmdlets (pronounced "command-lets") for interacting with databases, such as `Invoke-Sqlcmd` for executing SQL queries and `Import-Csv` for importing data from CSV files.

These are just a few examples of the languages used for database scripting. Other languages like Perl, Ruby, and JavaScript (with Node.js) can also be used depending on the specific requirements and environment. In the next section, we'll take a short look at one application involving Python.

## Object-Relational Modeling in Python using SQLAlchemy

In **Object-Oriented Programming (OOP)**, an **object** is a fundamental concept that represents a distinct entity with its own set of attributes and behaviors. Objects are used to model real-world entities or abstract concepts in a software system, providing a way to organize and structure code by grouping related data and behavior into a single unit. This "Object model of data" (very) common way of organizing data in many modern programming languages, including Java, Javascript, C++, and Python.

In the context of a music database, an object could represent a song, encapsulating attributes such as title, artist, album, and duration. These **attributes** define the state of the song object, while **methods** can be added to perform specific operations related to the song, such as playing or displaying its information. This is contrast to a table `song' in a relational database, which could store data, but which would not have any "methods" (instead, we interact with the data using SQL).

**Object-Relational Mapping (ORM)** is a technique that bridges the gap between the object-oriented paradigm (great for programming!) and the relational database model (great for storing data!). It allows developers to interact with databases using objects and object-oriented principles, rather than writing raw SQL queries.  Common tools for ORM include **Hibernate** and **Ebean** (both for the Java programming language), the **Entity Framework** (for C#), and SQLAlchemy (for Python--see below).

**SQLAlchemy** is a popular and feature-rich ORM library for Python. It provides a set of tools and abstractions for working with databases in an object-oriented manner, supporting various database backends such as SQLite, MySQL, PostgreSQL, and more.  With SQLAlchemy, you define Python classes that represent database tables, and the ORM takes care of mapping those classes to the corresponding tables in the database. Each class becomes a mapped entity, and instances of those classes represent individual rows in the table.

Let's dive into an example that demonstrates object-relational modeling using SQLAlchemy in the context of a music database, this time focusing on a songs table.

### Example: Modeling the Songs Table using SQLAlchemy

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import sessionmaker, declarative_base
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("sqlalchemy.engine.Engine.myengine").setLevel(logging.INFO)

# Create a connection to the database
engine = create_engine('sqlite:///songs.db',  logging_name = "myengine")

# Create a session factory
Session = sessionmaker(bind=engine)

# Create a base class for declarative models
Base = declarative_base()

# Define the Song class
class Song(Base):
    __tablename__ = 'songs'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist = Column(String)
    album = Column(String)
    duration = Column(Float)

    def __repr__(self):
        return f"<Song(id={self.id}, title='{self.title}', artist='{self.artist}', album='{self.album}', duration={self.duration})>"

# Create the tables in the database
Base.metadata.create_all(engine)

INFO:sqlalchemy.engine.Engine.myengine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine.myengine:PRAGMA main.table_info("songs")
INFO:sqlalchemy.engine.Engine.myengine:[raw sql] ()
INFO:sqlalchemy.engine.Engine.myengine:PRAGMA temp.table_info("songs")
INFO:sqlalchemy.engine.Engine.myengine:[raw sql] ()
INFO:sqlalchemy.engine.Engine.myengine:
CREATE TABLE songs (
	id INTEGER NOT NULL, 
	title VARCHAR, 
	artist VARCHAR, 
	album VARCHAR, 
	duration FLOAT, 
	PRIMARY KEY (id)
)


INFO:sqlalchemy.engine.Engine.myengine:[no key 0.00142s] ()
INFO:sqlalchemy.engine.Engine.myengine:COMMIT


This code demonstrates a basic usage of SQLAlchemy, which is an Object-Relational Mapping (ORM) library for Python. Here's a high-level overview of what the code does:

1.  It starts by importing the necessary modules from SQLAlchemy: `create_engine`, `Column`, `Integer`, `String`, `Float`, `sessionmaker`, and `declarative_base`.
2.  It configures logging to display information about the database operations.
3.  It creates a connection to an SQLite database named "songs.db" using `create_engine()`. The `logging_name` parameter is set to "myengine" to identify the specific engine instance in the logs.
4.  It creates a session factory using `sessionmaker()`, which is bound to the database engine. This factory will be used to create sessions for interacting with the database.
5.  It creates a base class for declarative models using `declarative_base()`. This base class will be used to define the database models.
6.  It defines a `Song` class that inherits from the base class. The `Song` class represents a table named "songs" in the database and has the following columns:
    -   `id`: An integer column serving as the primary key.
    -   `title`: A string column for the song title.
    -   `artist`: A string column for the artist name.
    -   `album`: A string column for the album name.
    -   `duration`: A float column for the song duration.The `__repr__()` method is defined to provide a string representation of the `Song` object.
7.  Finally, it creates the tables in the database using `Base.metadata.create_all(engine)`. This statement generates the necessary SQL statements to create the "songs" table based on the defined `Song` class.

The output you see is the logging information generated by SQLAlchemy. It shows the SQL statements being executed to check the existence of the "songs" table and create it if it doesn't exist. The output indicates that the table was successfully created with the specified columns.

Next, let's create some Song objects and perform database operations:

In [2]:
# Create a new session
session = Session()

# Create song objects
song1 = Song(title='Misery Business', artist='Paramore', album='Riot!', duration=3.31)
song2 = Song(title='Kyoto', artist='Phoebe Bridgers', album='Punisher', duration=3.04)
song3 = Song(title='Wish You Were Here', artist='Pink Floyd', album='Wish You Were Here', duration=5.40)

# Add the songs to the session
session.add_all([song1, song2, song3])
session.commit()

INFO:sqlalchemy.engine.Engine.myengine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine.myengine:INSERT INTO songs (title, artist, album, duration) VALUES (?, ?, ?, ?) RETURNING id
INFO:sqlalchemy.engine.Engine.myengine:[generated in 0.00033s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('Misery Business', 'Paramore', 'Riot!', 3.31)
INFO:sqlalchemy.engine.Engine.myengine:INSERT INTO songs (title, artist, album, duration) VALUES (?, ?, ?, ?) RETURNING id
INFO:sqlalchemy.engine.Engine.myengine:[insertmanyvalues 2/3 (ordered; batch not supported)] ('Kyoto', 'Phoebe Bridgers', 'Punisher', 3.04)
INFO:sqlalchemy.engine.Engine.myengine:INSERT INTO songs (title, artist, album, duration) VALUES (?, ?, ?, ?) RETURNING id
INFO:sqlalchemy.engine.Engine.myengine:[insertmanyvalues 3/3 (ordered; batch not supported)] ('Wish You Were Here', 'Pink Floyd', 'Wish You Were Here', 5.4)
INFO:sqlalchemy.engine.Engine.myengine:COMMIT


In this code chunk, we create instances of the `Song` class, representing individual songs. We set the attributes (`title`, `artist`, `album`, `duration`) for each song object.

We then add the song objects to the session using `session.add_all()` and commit the changes to the database using `session.commit()`. This operation inserts the song records into the `songs` table.

Again, you can see the underlying SQL code, which consists of some INSERT statements. (Later, we'll learn more about why we need to question marks --`?`--when interacting with databases in this way for security reasons).

Finally, let's query the database to retrieve the songs.

In [3]:
# query the songs table
songs = session.query(Song).all()
for song in songs:
    print(song)

INFO:sqlalchemy.engine.Engine.myengine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine.myengine:SELECT songs.id AS songs_id, songs.title AS songs_title, songs.artist AS songs_artist, songs.album AS songs_album, songs.duration AS songs_duration 
FROM songs
INFO:sqlalchemy.engine.Engine.myengine:[generated in 0.00282s] ()


<Song(id=1, title='Misery Business', artist='Paramore', album='Riot!', duration=3.31)>
<Song(id=2, title='Kyoto', artist='Phoebe Bridgers', album='Punisher', duration=3.04)>
<Song(id=3, title='Wish You Were Here', artist='Pink Floyd', album='Wish You Were Here', duration=5.4)>


Object-relational modeling with SQLAlchemy (and similar tools) provides a powerful and intuitive way to interact with databases in Python. By defining classes that represent database tables and leveraging SQLAlchemy's ORM capabilities, we can work with data in an object-oriented manner, abstracting away the complexities of database queries.

SQLAlchemy's declarative syntax allows us to define the structure of our database tables using Python classes, making the code more readable and maintainable. The ORM takes care of generating the appropriate SQL statements behind the scenes, enabling developers to focus on the application logic and business requirements.

However, it's important to keep in mind that while ORMs like SQLAlchemy simplify database interactions, it's still crucial to have a good understanding of SQL and database concepts. Reviewing the generated SQL queries and optimizing performance when necessary is essential for building efficient and scalable applications. Importantly, there is no guarantee the ORMs will generate the best-performing query, or that they can handle the logic of especially complex or detailed queries.

## Key Points
-   The INSERT INTO statement is used to add new rows of data to SQL tables, supporting both single-row and multiple-row insertions.
-   Constraints, such as primary keys, foreign keys, and CHECK constraints, ensure data integrity and consistency.
-   Constraint violations during data insertion can be handled by modifying the data or updating the table structure.
-   Auto-incrementing primary keys simplify data insertion by automatically assigning unique identifiers to new rows.
-   Subqueries can be used in INSERT statements to dynamically insert data based on existing records.-   The DELETE statement removes rows from a table based on specified conditions, while DROP TABLE removes the entire table structure.
-   "Soft delete" involves marking records as deleted instead of permanently removing them, allowing for data retention and historical tracking.
-   The UPDATE statement modifies existing data in a table based on specified conditions.
-   Triggers automatically execute specific actions before or after database events, such as INSERT, UPDATE, or DELETE operations.
-   Logging mechanisms can be implemented using triggers to capture and store changes made to database tables for auditing and monitoring purposes.
-   Database scripting using Linux scripts and programming languages like Bash, Python, and PowerShell enables automation and flexibility in database tasks.
-   Automating database tasks through scripting improves efficiency, reduces manual errors, and enhances productivity in database management.